In [3]:
import ast
import os
import time

# internal imports
import pickle as pkl

import tensorflow as tf
import magenta

import polyphony_model
import polyphony_sequence_generator
import polyphony_lib
import polyphony_encoder_decoder

from  sequence_example_lib import *

from magenta.music import constants
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2

import magenta.music as mm

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
def to_events(x):
    return np.argmax(x, axis=1)
def to_real_length(x):
    while(x[len(x)-1]==0):
        x.pop()
    # delete the last note along with padded zeros
    x.pop()

In [ ]:
def sequence_example_to_real_inputs(sequence_example_path):
    # train
    sequence_example_file = sequence_example_path
    sequence_example_file_paths = tf.gfile.Glob(
        os.path.expanduser(sequence_example_file))
    start_time = time.time()
    inputs, labels, lengths = get_numpy_from_tf_sequence_example( input_size=259,
                                        sequence_example_file_paths = sequence_example_file_paths,
                                        shuffle = False)
    print('Time:',time.time() - start_time)
    print('inputs shape',inputs.shape)
    print('inputs type',type(inputs))
    input_events = []
    for i in inputs:
        input_events.append(to_events(i))
    real_inputs = []
    for i in input_events:
        d = []
        d = list(i)
        to_real_length(d)
        real_inputs.append(d)
    return real_inputs

In [ ]:
def polyphony_sequence_example_to_list_pkl(sequence_example_path, output_dir, name):
    real_inputs = sequence_example_to_real_inputs(sequence_example_path)
    real_inputs_path = os.path.join(output_dir, name+'.pkl')
    with open(real_inputs_path, 'wb') as mf:   #pickle只能以二进制格式存储数据到文件
        mf.write(pkl.dumps(real_inputs) )   #dumps序列化源数据后写入文件
        mf.close()
        print("Write %s to %s" % (real_inputs_path, output_dir) )

In [ ]:
poly_S_E_path = '/Users/mac/Desktop/MusicGeneration/Mag_Data/Poly_S_E/eval_poly_tracks.tfrecord'
polyphony_sequence_example_to_list_pkl(poly_S_E_path, '/Users/mac/Desktop/test', 'test_list')

In [ ]:
test_list_path = '/Users/mac/Desktop/test/test_list.pkl'
with open(test_list_path, 'rb') as tl_file:
    test_list = pkl.load(tl_file)
    tl_file.close()

In [24]:
def midi_to_polyphony_sequence(steps_per_quarter, qpm, midi_dir, midi_name):
    primer_midi = os.path.join(midi_dir, midi_name)
    primer_midi = os.path.expanduser(primer_midi)

    primer_sequence = magenta.music.midi_file_to_sequence_proto(primer_midi)
    if primer_sequence.tempos and primer_sequence.tempos[0].qpm:
        qpm = primer_sequence.tempos[0].qpm
        
    # Quantize the priming sequence.
    quantized_primer_sequence = mm.quantize_note_sequence(
        primer_sequence, steps_per_quarter)

    extracted_seqs, _ = polyphony_lib.extract_polyphonic_sequences(
        quantized_primer_sequence, start_step=0)
    
    poly_seq = extracted_seqs[0]
    
    return poly_seq

In [25]:
test_poly = midi_to_polyphony_sequence(4, 160, '/Users/mac/Desktop/test', 'test0.mid')

In [26]:
def polyphony_sequence_to_list(poly_seq):
    ret = []
    poly_enc_dec = polyphony_encoder_decoder.PolyphonyOneHotEncoding()
    for poly_event in poly_seq:
        ret.append(poly_enc_dec.encode_event(poly_event))
    return ret

In [27]:
test_list = polyphony_sequence_to_list(test_poly)

In [28]:
print(test_list)

[0, 2, 2, 2, 2, 2, 2, 56, 52, 49, 2, 2, 63, 57, 54, 49, 2, 191, 185, 182, 177, 2, 191, 185, 182, 177, 2, 2, 63, 56, 54, 48, 2, 191, 184, 182, 176, 2, 191, 184, 182, 176, 2, 191, 184, 182, 176, 2, 191, 184, 182, 176, 2, 2, 63, 56, 54, 48, 2, 191, 184, 182, 176, 2, 64, 56, 52, 49, 2, 192, 184, 180, 177, 2, 192, 184, 180, 177, 2, 192, 184, 177, 2, 61, 57, 52, 45, 2, 189, 185, 180, 173, 2, 189, 185, 180, 173, 2, 189, 185, 180, 173, 2, 2, 2, 61, 2, 189, 2, 69, 61, 57, 51, 42, 2, 197, 189, 185, 179, 170, 2, 197, 189, 185, 179, 170, 2, 179, 170, 2, 68, 60, 56, 51, 44, 2, 196, 188, 184, 179, 172, 2, 196, 188, 184, 179, 172, 2, 66, 64, 188, 184, 179, 172, 2, 66, 188, 184, 2, 194, 188, 184, 2, 194, 188, 184, 2, 194, 63, 188, 2, 64, 61, 56, 49, 2, 192, 189, 184, 177, 2, 177, 2, 51, 2, 56, 52, 49, 2, 184, 180, 177, 2, 184, 180, 177, 2, 56, 54, 177, 2, 57, 2, 185, 2, 185, 2, 56, 52, 49, 2, 63, 57, 54, 49, 2, 191, 185, 182, 177, 2, 182, 177, 2, 52, 177, 2, 63, 56, 54, 48, 2, 191, 184, 182, 176, 2, 1

In [ ]:
def list_to_midi(events, qpm, output_dir, midi_name):
    poly_seq = polyphony_lib.PolyphonicSequence(steps_per_quarter=4)
    pe = polyphony_lib.PolyphonicEvent
    poly_enc_dec = polyphony_encoder_decoder.PolyphonyOneHotEncoding()
    poly_events = []
    for event in events:
        poly_events.append(poly_enc_dec.decode_event(event))
    
    for event in poly_events:
        poly_seq.append(event)

    poly_seq_ns = pol y_seq.to_sequence(qpm=qpm)
    
    midi_path = os.path.join(output_dir, midi_name+'.mid')
    magenta.music.sequence_proto_to_midi_file(poly_seq_ns, midi_path)

    print('Wrote %s.mid to %s'%(midi_name, output_dir) )

In [ ]:
print(test_list[0])

In [ ]:
events = [0,2, 2, 71, 2, 70, 2, 71, 2, 73, 2, 74, 2, 66, 2, 67, 2, 69, 2, 67, 2, 64, 2, 76, 2, 74, 2, 73, 2, 71, 2, 70, 2, 79, 2, 78, 2, 76, 2, 74, 2, 73, 2, 71, 2, 70, 2, 71, 2, 73, 2, 71, 2, 73, 2, 74, 2, 74, 73, 73, 2, 74, 2, 73, 73, 71, 2, 74, 2, 73, 2, 71, 59, 2, 73, 58, 2, 74, 59, 2, 76, 61, 2, 78, 62, 2, 206, 54, 2, 79, 78, 55, 2, 78, 57, 2, 76, 55, 2, 204, 52, 2, 64, 2, 62, 2, 76, 61, 2, 204, 59, 2, 77, 76, 58, 2, 76, 67, 2, 74, 66, 2, 202, 64, 2, 62, 2, 61, 2, 73, 59, 2, 201, 58, 2, 74, 59, 2, 202, 61, 2, 67, 59, 2, 195, 61, 2, 195, 62, 2, 64, 62, 61, 2, 69, 62, 61, 61, 2, 64, 59, 2, 66, 62, 61, 2, 194, 61, 2, 74, 59, 2, 202, 61, 2, 202, 62, 2, 73, 64, 2, 74, 66, 2, 69, 194, 2, 71, 67, 66, 2, 74, 66, 2, 79, 64, 2, 78, 192, 2, 76, 2, 74, 2, 73, 64, 2, 71, 192, 2, 69, 66, 64, 2, 73, 64, 2, 78, 62, 2, 76, 190, 2, 74, 2, 73, 2, 71, 61, 2, 69, 189, 2, 67, 62, 2, 71, 190, 2, 76, 55, 2, 71, 183, 2, 73, 183, 2, 201, 52, 2, 201, 57, 2, 73, 52, 2, 74, 54, 2, 69, 182, 2, 71, 62, 2, 69, 190, 2, 78, 190, 2, 206, 61, 2, 206, 62, 2, 206, 57, 2, 206, 59, 2, 69, 62, 2, 71, 67, 2, 69, 66, 2, 79, 64, 2, 207, 62, 2, 207, 61, 2, 207, 59, 2, 207, 57, 2, 69, 61, 2, 74, 66, 2, 73, 64, 2, 78, 62, 2, 76, 61, 2, 79, 59, 2, 78, 57, 2, 83, 55, 2, 81, 59, 2, 79, 64, 2, 78, 62, 2, 76, 62, 61, 2, 81, 61, 2, 79, 189, 2, 81, 61, 2, 78, 62, 2, 81, 57, 2, 79, 59, 2, 81, 57, 2, 74, 66, 2, 78, 194, 2, 76, 194, 2, 78, 194, 2, 71, 194, 2, 73, 57, 2, 74, 59, 2, 76, 57, 2, 73, 67, 2, 74, 195, 2, 76, 195, 2, 78, 195, 2, 74, 195, 2, 76, 57, 2, 78, 62, 2, 79, 61, 2, 81, 66, 2, 79, 64, 2, 83, 67, 2, 81, 66, 2, 79, 71, 2, 78, 69, 2, 76, 67, 2, 79, 66, 2, 73, 64, 2, 201, 69, 2, 201, 67, 2, 201, 69, 2, 201, 66, 2, 73, 194, 2, 78, 50, 2, 73, 52, 2, 74, 54, 2, 71, 56, 2, 73, 57, 2, 69, 49, 2, 71, 50, 2, 199, 52, 2, 199, 50, 2, 199, 47, 2, 199, 59, 2, 78, 57, 2, 77, 56, 2, 80, 54, 2, 71, 53, 2, 69, 62, 2, 71, 61, 2, 199, 59, 2, 199, 57, 2, 77, 56, 2, 75, 54, 2, 73, 53, 2, 81, 54, 2, 209, 56, 2, 209, 54, 2, 75, 56, 2, 77, 57, 56, 2, 205, 57, 2, 205, 57, 56, 2, 78, 56, 54, 2, 78, 57, 56, 2, 206, 56, 2, 66, 54, 2, 65, 56, 2, 66, 57, 2, 68, 59, 2, 69, 61, 2, 189, 61, 2, 62, 61, 2, 64, 189, 2, 62, 59, 2, 187, 59, 2, 71, 2, 69, 2, 68, 59, 2, 66, 187, 2, 65, 61, 59, 2, 74, 59, 2, 73, 57, 2, 71, 185, 2, 69, 2, 68, 2, 66, 56, 2, 65, 184, 2, 66, 57, 2, 68, 185, 2, 66, 50, 2, 68, 178, 2, 69, 178, 2, 69, 68, 47, 2, 69, 68, 52, 2, 68, 68, 66, 47, 2, 69, 49, 2, 68, 177, 2, 66, 57, 2, 68, 185, 2, 69, 185, 2, 71, 56, 2, 73, 57, 2, 201, 52, 2, 74, 73, 54, 2, 73, 57, 2, 71, 62, 2, 199, 61, 2, 59, 2, 57, 2, 71, 56, 2, 199, 54, 2, 73, 71, 52, 2, 71, 56, 2, 69, 61, 2, 197, 59, 2, 57, 2, 56, 2, 68, 54, 2, 196, 52, 2, 69, 50, 2, 197, 54, 2, 62, 59, 2, 190, 54, 2, 190, 57, 56, 2, 59, 56, 2, 64, 184, 2, 59, 56, 2, 61, 57, 2, 189, 52, 2, 69, 54, 2, 197, 52, 2, 197, 61, 2, 68, 189, 2, 69, 189, 2, 64, 189, 2, 66, 189, 2, 69, 52, 2, 74, 54, 2, 73, 52, 2, 71, 62, 2, 69, 190, 2, 68, 190, 2, 66, 190, 2, 64, 190, 2, 68, 52, 2, 73, 57, 2, 71, 56, 2, 69, 61, 2, 68, 59, 2, 66, 62, 2, 64, 61, 2, 66, 62, 2, 66, 64, 2, 71, 62, 2, 66, 61, 2, 68, 59, 2, 196, 64, 2, 196, 62, 2, 68, 64, 2, 69, 61, 2, 64, 2, 66, 62, 2, 64, 2, 73, 57, 2, 201, 61, 2, 201, 59, 2, 201, 61, 2, 201, 54, 2, 64, 56, 2, 66, 57, 2, 64, 59, 2, 74, 56, 2, 202, 57, 2, 202, 59, 2, 202, 61, 2, 202, 57, 2, 64, 59, 2, 69, 61, 2, 68, 62, 2, 73, 64, 2, 71, 62, 2, 74, 66, 2, 73, 64, 2, 78, 62, 2, 76, 61, 2, 74, 59, 2, 73, 62, 2, 71, 56, 2, 76, 184, 2, 74, 184, 2, 76, 184, 2, 73, 184, 2, 71, 49, 2, 69, 54, 2, 68, 53, 2, 73, 57, 2, 71, 56, 2, 74, 59, 2, 73, 57, 2, 71, 62, 2, 69, 61, 2, 68, 59, 2, 71, 57, 2, 76, 56, 2, 204, 61, 2, 204, 59, 2, 204, 61, 2, 204, 58, 2, 66, 64, 2, 71, 62, 2, 70, 61, 2, 74, 59, 2, 73, 57, 2, 76, 55, 2, 74, 54, 2, 79, 52, 2, 78, 50, 2, 76, 49, 2, 74, 47, 2, 73, 54, 2, 78, 182, 2, 76, 42, 2, 78, 170, 2, 74, 47, 2, 73, 175, 2, 71, 175, 2, 70, 49, 2, 71, 50, 2, 73, 52, 2, 74, 54, 2, 66, 182, 2, 67, 55, 54, 2, 69, 54, 2, 67, 52, 2, 64, 180, 2, 76, 2, 74, 2, 73, 52, 2, 71, 180, 2, 70, 54, 52, 2, 79, 52, 2, 78, 50, 2, 76, 178, 2, 74, 178, 2, 73, 52, 2, 71, 50, 2, 70, 49, 2, 71, 47, 2, 73, 45, 2, 71, 43, 2, 73, 42, 2, 74, 40, 2, 74, 73, 38, 2, 74, 73, 40, 2, 73, 71, 42, 2, 74, 73, 35, 2, 73, 163, 2, 71, 47, 2, 73, 46, 2, 74, 47, 2, 76, 49, 2, 78, 50, 2, 206, 42, 2, 79, 78, 43, 2, 78, 45, 2, 76, 43, 2, 204, 40, 2, 52, 2, 50, 2, 76, 49, 2, 204, 47, 2, 78, 76, 46, 2, 76, 55, 2, 74, 54, 2, 73, 52, 2, 78, 50, 2, 76, 49, 2, 79, 47, 2, 78, 46, 2, 78, 47, 2, 206, 42, 2, 206, 43, 2, 79, 40, 2, 74, 73, 42, 2, 74, 73, 73, 170, 2, 74, 73, 42, 2, 71, 170, 2, 71, 35, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 199, 163, 2, 1]
for i in range(10):
    list_to_midi(test_list[i], 120, '/Users/mac/Desktop/test', 'test%d'%i)